In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

%load_ext tensorboard

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

Xavier and He initialization

In [2]:
[name for name in dir(keras.initializers) if not name.startswith("_")]

['Constant',
 'GlorotNormal',
 'GlorotUniform',
 'HeNormal',
 'HeUniform',
 'Identity',
 'Initializer',
 'LecunNormal',
 'LecunUniform',
 'Ones',
 'Orthogonal',
 'RandomNormal',
 'RandomUniform',
 'TruncatedNormal',
 'VarianceScaling',
 'Zeros',
 'constant',
 'deserialize',
 'get',
 'glorot_normal',
 'glorot_uniform',
 'he_normal',
 'he_uniform',
 'identity',
 'lecun_normal',
 'lecun_uniform',
 'ones',
 'orthogonal',
 'random_normal',
 'random_uniform',
 'serialize',
 'truncated_normal',
 'variance_scaling',
 'zeros']

In [3]:
keras.layers.Dense(10, activation="relu", kernel_initializer="he_normal")

In [7]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [8]:
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.PReLU(),
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.PReLU(),
    keras.layers.Dense(10, activation="softmax")
])

In [9]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [10]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 7s 4ms/step - loss: 1.3008 - accuracy: 0.6271 - val_loss: 0.8807 - val_accuracy: 0.7330
Epoch 2/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.7906 - accuracy: 0.7483 - val_loss: 0.7018 - val_accuracy: 0.7814
Epoch 3/10
1719/1719 [==============================] - 6s 4ms/step - loss: 0.6767 - accuracy: 0.7789 - val_loss: 0.6380 - val_accuracy: 0.7968
Epoch 4/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.6193 - accuracy: 0.7972 - val_loss: 0.5852 - val_accuracy: 0.8110
Epoch 5/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.5818 - accuracy: 0.8076 - val_loss: 0.5542 - val_accuracy: 0.8180
Epoch 6/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.5549 - accuracy: 0.8144 - val_loss: 0.5323 - val_accuracy: 0.8284
Epoch 7/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.5344 - accuracy: 0.8204 - val_loss: 0.5138 - val_accuracy:

In [11]:
def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6) # sandals or shirts
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2 # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6], y_A),
            (X[y_5_or_6], y_B))

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [12]:
tf.random.set_seed(42)
np.random.seed(42)

In [13]:
model_A = keras.models.Sequential()
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="selu"))
model_A.add(keras.layers.Dense(8, activation="softmax"))

In [14]:
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=["accuracy"])

In [15]:
history = model_A.fit(X_train_A, y_train_A, epochs=20,
                    validation_data=(X_valid_A, y_valid_A))

Epoch 1/20
1375/1375 [==============================] - 6s 4ms/step - loss: 0.5820 - accuracy: 0.8146 - val_loss: 0.3878 - val_accuracy: 0.8612
Epoch 2/20
1375/1375 [==============================] - 5s 4ms/step - loss: 0.3598 - accuracy: 0.8763 - val_loss: 0.3318 - val_accuracy: 0.8799
Epoch 3/20
1375/1375 [==============================] - 5s 4ms/step - loss: 0.3222 - accuracy: 0.8885 - val_loss: 0.3054 - val_accuracy: 0.8914
Epoch 4/20
1375/1375 [==============================] - 5s 4ms/step - loss: 0.3024 - accuracy: 0.8956 - val_loss: 0.2919 - val_accuracy: 0.8991
Epoch 5/20
1375/1375 [==============================] - 5s 4ms/step - loss: 0.2888 - accuracy: 0.9005 - val_loss: 0.2794 - val_accuracy: 0.9051
Epoch 6/20
1375/1375 [==============================] - 5s 3ms/step - loss: 0.2786 - accuracy: 0.9053 - val_loss: 0.2736 - val_accuracy: 0.9043
Epoch 7/20
1375/1375 [==============================] - 5s 4ms/step - loss: 0.2701 - accuracy: 0.9079 - val_loss: 0.2707 - val_accuracy:

In [16]:
model_A.save("my_model_A.h5")

C:\Users\ma6114132\AppData\Roaming\Python\Python39\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [17]:
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_B.add(keras.layers.Dense(n_hidden, activation="selu"))
model_B.add(keras.layers.Dense(1, activation="sigmoid"))

In [18]:
model_B.compile(loss="binary_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=["accuracy"])

In [19]:
history = model_B.fit(X_train_B, y_train_B, epochs=20,
                      validation_data=(X_valid_B, y_valid_B))

Epoch 1/20
7/7 [==============================] - 1s 60ms/step - loss: 0.7247 - accuracy: 0.5100 - val_loss: 0.6089 - val_accuracy: 0.6988
Epoch 2/20
7/7 [==============================] - 0s 22ms/step - loss: 0.5681 - accuracy: 0.7600 - val_loss: 0.5093 - val_accuracy: 0.8215
Epoch 3/20
7/7 [==============================] - 0s 22ms/step - loss: 0.4736 - accuracy: 0.8350 - val_loss: 0.4406 - val_accuracy: 0.8671
Epoch 4/20
7/7 [==============================] - 0s 21ms/step - loss: 0.4067 - accuracy: 0.8850 - val_loss: 0.3881 - val_accuracy: 0.8955
Epoch 5/20
7/7 [==============================] - 0s 21ms/step - loss: 0.3553 - accuracy: 0.9100 - val_loss: 0.3458 - val_accuracy: 0.9280
Epoch 6/20
7/7 [==============================] - 0s 22ms/step - loss: 0.3130 - accuracy: 0.9400 - val_loss: 0.3125 - val_accuracy: 0.9412
Epoch 7/20
7/7 [==============================] - 0s 23ms/step - loss: 0.2802 - accuracy: 0.9550 - val_loss: 0.2848 - val_accuracy: 0.9493
Epoch 8/20
7/7 [===========